# Threading

alternative to multithreading using the threading module. World should come before dont know why

In [2]:
import time
from threading import Thread


def greet():
    time.sleep(3)
    print('Hello, ')


t = Thread(target=greet())
t.start()

print('world!')

Hello, 
world!


An example with multipke threads:

In [3]:
import time
from threading import Thread


def cube_area(thread, length, delay=0):
    time.sleep(delay)
    print(f"{thread} ---> Area of a cube with an edge length of {length} is: \t{6 * (length ** 2)}")


def circle_area(thread, length, delay=0):
    time.sleep(delay)
    print(f"{thread} ---> Area of a circle with a radius length of {length} is: \t{3.14 * (length ** 2)}")

# instantiate multiple threads with functions as targets and thread name, length as arguments

t1 = Thread(target=cube_area, args=("t1", 2))
t2 = Thread(target=circle_area, args=("t2", 3))

t3 = Thread(target=cube_area, args=("t3", 4))
t4 = Thread(target=circle_area, args=("t4", 6))

t5 = Thread(target=cube_area, args=("t5", 9))
t6 = Thread(target=circle_area, args=("t6", 8))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()
t6.start()

t1 ---> Area of a cube with an edge length of 2 is: 	24
t2 ---> Area of a circle with a radius length of 3 is: 	28.26
t3 ---> Area of a cube with an edge length of 4 is: 	96
t4 ---> Area of a circle with a radius length of 6 is: 	113.04
t5 ---> Area of a cube with an edge length of 9 is: 	486
t6 ---> Area of a circle with a radius length of 8 is: 	200.96


In [5]:
import time
from threading import Thread


def cube_area(thread, length, delay=0):
    time.sleep(delay)
    print(f"{thread} ---> Area of a cube with an edge length of {length} is: \t{6 * (length ** 2)}")


def circle_area(thread, length, delay=0):
    time.sleep(delay)
    print(f"{thread} ---> Area of a circle with a radius length of {length} is: \t{3.14 * (length ** 2)}")

# instantiate multiple threads with functions as targets and thread name, length as arguments
t1 = Thread(target=cube_area, args=("t1", 2, 3))
t2 = Thread(target=circle_area, args=("t2", 2, 2))

t3 = Thread(target=cube_area, args=("t3", 4, 1))
t4 = Thread(target=circle_area, args=("t4", 6, 2))

t5 = Thread(target=cube_area, args=("t5", 9, 4))
t6 = Thread(target=circle_area, args=("t6", 8, 3))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()
t6.start()

## Race condition

Lets first see an example of why we need race condition. in the below application, each dread print the items name and price. The output shows us which thread was running. Each time we are running the application, the result depends on which thread is running first.

In [6]:
import time
from threading import Thread

total = 0


def calc_price(name, item_price):
    for i in range(3):
        print("Item: ", name)
        time.sleep(2)
        total = item_price
        print("Price: ", total)


t1 = Thread(target=calc_price, args=("Shirt", 5))
t2 = Thread(target=calc_price, args=("Jeans", 10))

t1.start()
t2.start()

Item:  Shirt
Item:  Jeans
Price:  5
Item:  Shirt
Price:  10
Item:  Jeans
Price:  10
Item:  Jeans
Price:  5
Item:  Shirt
Price:  5
Price:  10


To solve this problem we need to sync the threads. To do this we will use lock, locking access to a cerstain function. In a Lock object, only one thread at a time is allowed to execute, but occasionally, we need to execute a particular number of threads simultaneously.

In [7]:
from threading import Thread, Lock
import time

l = Lock()
total = 0


def calc_price(name, item_price):
    for i in range(3):
        l.acquire()
        print("Item:", name)
        time.sleep(2)
        total = item_price
        print("Price:", total)
        l.release()


t1 = Thread(target=calc_price, args=("Shirt", 5))
t2 = Thread(target=calc_price, args=("Jeans", 10))

t1.start()
t2.start()

Item: Shirt


We can wait for a thread to finish the execution by calling the join() function. This method allows the current thread to be blocked until the target thread it has joined is finished.

In [ ]:
t1.start()
t1.join()

t2.start()
t2.join()

Let's modify this example and add five items and prices, each represented with its thread. Also, let's change it, so three threads can simultaneously access the total price. In this case, we cannot use Lock anymore. We should go further and use the concept of the semaphore.

The semaphore concept is one of the oldest synchronization primitives in the history of computer science, invented by the early Dutch computer scientist Edsger W. Dijkstra. He used the names `P()` and `V()` instead of `acquire()` and `release()`.

Semaphores can be of two types:
1. Binary Semaphore — this semaphore can have only two values – 0 or 1. Its value is initialized to 1.
2. Counting Semaphore — its value can be 0, 1, or other integer values. It is used to control access to a resource that has multiple instances.


In [9]:
from threading import Thread, Semaphore
import time

# creating Semaphore, where count = 3
sem = Semaphore(3)
total = 0


def calc_price(name, item_price):
    sem.acquire()
    for i in range(2):
        print("Item:", name)
        time.sleep(10)
        total = item_price
        print("Price:", total)
        sem.release()


# creating multiple threads
t1 = Thread(target=calc_price, args=("Shirt", 5))
t2 = Thread(target=calc_price, args=("Jeans", 10))
t3 = Thread(target=calc_price, args=("Dress", 12))
t4 = Thread(target=calc_price, args=("Belt", 3))
t5 = Thread(target=calc_price, args=("Bag", 20))

# calling the threads
t1.start()
t2.start()
t3.start()
t4.start()
t5.start()

Item: Shirt
Item: Jeans
Item: Dress


As with files, you can also use the with context manager with a semaphore. It will allow you to omit the explicit calling of acquire() and release(), since it will be automatically managed by the manager. calc_price() may also look this way:

In [ ]:
def calc_price(name, item_price):
    with sem:
        for i in range(2):
            print("Item:", name)
            time.sleep(10)
            total = item_price
            print("Price:", total)

In this example, we've created an instance of the `Semaphore` class, called `sem` where the value of `count` is 3. This means that three threads can access sem at a time.

Whenever we call the `start()` method, three threads are allowed to access the semaphore, and hence three threads are allowed to execute `calc_price()` method at a time.